In [1]:
import gograph
import similarity
import importlib
import re
import time
import annotations

import pandas as pd
import numpy as np

In [2]:
def reload_my_modules():
    """Re-import my modules into the IPython shell"""
    importlib.reload(annotations)
    importlib.reload(gograph)
    importlib.reload(similarity)

In [4]:
%%time
reload_my_modules()
fp = 'data/goa_human.gaf'
anno = annotations.Annotations(fp)

/Users/ilyanovikov/anaconda3/lib/python3.6/site-packages/IPython/core/magic.py:187: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  call = lambda f, *a, **k: f(*a, **k)


CPU times: user 1.49 s, sys: 177 ms, total: 1.66 s
Wall time: 1.7 s


In [5]:
# num proteins
print(anno.annotations.groupby('DB_Object_ID').size().count())
# num annos per protein
print(anno.annotations.groupby('DB_Object_ID').size().mean())
#
print(anno.annotations.groupby('DB_Object_ID').size().quantile([0, 0.25, 0.5, 0.75, 1]))
# total annos
print(len(anno.annotations))

19626
25.621216753286458
0.00      1.0
0.25      7.0
0.50     15.0
0.75     30.0
1.00    914.0
dtype: float64
502842


In [7]:
%%time
reload_my_modules()
fp = 'data/go-basic.obo'
#fp = 'data/gene_ontology_edit.obo.2019-07-01'
ontology = gograph.OBOParser(fp).parse_ontology()

CPU times: user 1.99 s, sys: 86.5 ms, total: 2.08 s
Wall time: 2.12 s


In [8]:
t = time.time()
ontology.make_ancestry_matrix()
print(time.time()-t)

8.442905902862549


In [9]:
ontology.ancestry_matrix.matrix.setdiag(1)

In [10]:
anno.annotations = anno.annotations.loc[anno.annotations.Aspect == 'P', :]

In [11]:
len(anno.annotations)

164407

In [12]:
importlib.reload(annotations)
ft = annotations.FrequencyTable(anno, ontology.ancestry_matrix)

/Users/ilyanovikov/Documents/CodeRepos/ggid/annotations.py:52: RuntimeWarning: divide by zero encountered in log10
  self.information_content = -1*np.log10(self.deep_count.sum(axis=0)/self.deep_count.sum())


In [16]:
p = pd.read_csv('data/list_of_human_kinases.csv')
p.head()

,Unnamed: 0,gene_symbol,uniprot
0,0,AKT1,P31749
1,1,AKT2,P31751
2,2,AKT3,Q9Y243
3,3,MRCKA,Q5VT25
4,4,MRCKB,Q9Y5S2


In [17]:
reload_my_modules()

#P25098 GRK2
#P35626 GRK3

sim = similarity.Similarity(anno, ontology, ft, list(p.uniprot), namespace='P', verbose=True)

511


In [18]:
sim.calculate_similarity_mult_cpu()

worker started
worker started
worker done 114.53437519073486
worker done 114.82369208335876


In [21]:
print(sim.workers)

[<Process(Process-1, started)>, <Process(Process-2, started)>]


In [24]:
t0 = time.time()
sim.calculate_similarity()
print(time.time() - t0)
print(sim.counter)

62.19319033622742
62941443


In [97]:
smt = sim.sim_matrix.todense()

In [98]:
from scipy import sparse

In [99]:
smt2=smt+smt.T

In [100]:
mask = smt2.argsort(axis=0) < 2

In [101]:
smt2[~mask] = 0

In [102]:
smt3 = sparse.coo_matrix(smt2)

In [103]:
smt3.nonzero()[1]

array([461, 157, 323, ..., 305,   0,   1], dtype=int32)

In [104]:
import pickle

In [105]:
pickle.dump(smt3, open('results/kinase_matrix.pkl', 'wb'))

In [106]:
abc = pickle.load(open('results/kinase_matrix.pkl', 'rb'))

In [107]:
abc.nonzero()

(array([ 41,  46,  50, ..., 495, 510, 510], dtype=int32),
 array([461, 157, 323, ..., 305,   0,   1], dtype=int32))

In [108]:
a, b = abc.nonzero()

In [109]:
print(len(a))

1022
